In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import scale
from keras.models import Sequential
from keras.layers import Dense, Dropout

Using TensorFlow backend.


In [2]:
path_train = "UJIndoorLoc/trainingData.csv"
path_validation = "UJIndoorLoc/validationData.csv"

In [3]:
#Explicitly pass header=0 to be able to replace existing names 
train_df = pd.read_csv(path_train,header = 0)
train_df = train_df[:19930]
train_AP_strengths = train_df.ix[:,:520] #select first 520 columns

#Scale transforms data to center to the mean and component wise scale to unit variance
train_AP_features = scale(np.asarray(train_AP_strengths))

#The following two objects are actually pandas.core.series.Series objects
building_ids_str = train_df["BUILDINGID"].map(str) #convert all the building ids to strings
building_floors_str = train_df["FLOOR"].map(str) #convert all the building floors to strings

res = building_ids_str + building_floors_str #element wise concatenation of BUILDINGID+FLOOR
train_labels = np.asarray(building_ids_str + building_floors_str)

#convert labels to categorical variables, dummy_labels has type 'pandas.core.frame.DataFrame'
dummy_labels = pd.get_dummies(train_labels)


"""one hot encode the dummy_labels.
this is done because dummy_labels is a dataframe with the labels (BUILDINGID+FLOOR) 
as the column names
"""
train_labels = np.asarray(dummy_labels) #labels is an array of shape 19937 x 13. (there are 13 types of labels)

c:\python\python35\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  after removing the cwd from sys.path.
c:\python\python35\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, _DataConversionWarning)


In [4]:
#generate len(train_AP_features) of floats in between 0 and 1
train_val_split = np.random.rand(len(train_AP_features))
#convert train_val_split to an array of booleans: if elem < 0.7 = true, else: false
train_val_split = train_val_split < 0.70 #should contain ~70% percent true

We aren't given a formal testing set, so we will treat the given validation set as the testing set

In [5]:
# We will then split our given training set into training + validation 
train_X = train_AP_features[train_val_split]
train_y = train_labels[train_val_split]
val_X = train_AP_features[~train_val_split]
val_y = train_labels[~train_val_split]

In [6]:
#Turn the given validation set into a testing set
test_df = pd.read_csv(path_validation,header = 0)
test_AP_features = scale(np.asarray(test_df.ix[:,0:520]))
test_labels = np.asarray(test_df["BUILDINGID"].map(str) + test_df["FLOOR"].map(str))
test_labels = np.asarray(pd.get_dummies(test_labels))

c:\python\python35\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  This is separate from the ipykernel package so we can avoid doing imports until
c:\python\python35\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, _DataConversionWarning)


In [7]:
nb_epochs = 20
batch_size = 10
input_size = 520
num_classes = 13

In [8]:
def encoder():
    model = Sequential()
    model.add(Dense(256, input_dim=input_size, activation='tanh', bias=True))
    model.add(Dense(128, activation='tanh', bias=True))
    model.add(Dense(64, activation='tanh', bias=True))
    return model

In [9]:
def decoder(e):   
    e.add(Dense(128, input_dim=64, activation='tanh', bias=True))
    e.add(Dense(256, activation='tanh', bias=True))
    e.add(Dense(input_size, activation='tanh', bias=True))
    e.compile(optimizer='adam', loss='mse')
    return e
    

In [10]:
e = encoder()

c:\python\python35\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(256, use_bias=True, input_dim=520, activation="tanh")`
  This is separate from the ipykernel package so we can avoid doing imports until
c:\python\python35\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, use_bias=True, activation="tanh")`
  after removing the cwd from sys.path.
c:\python\python35\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(64, use_bias=True, activation="tanh")`
  """


In [11]:
d = decoder(e)

c:\python\python35\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, use_bias=True, input_dim=64, activation="tanh")`
  
c:\python\python35\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(256, use_bias=True, activation="tanh")`
  This is separate from the ipykernel package so we can avoid doing imports until
c:\python\python35\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(520, use_bias=True, activation="tanh")`
  after removing the cwd from sys.path.


In [12]:
d.fit(train_X, train_X, nb_epoch=nb_epochs, batch_size=batch_size)

c:\python\python35\lib\site-packages\keras\models.py:851: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/20
13892/13892 [==============================] - 5s - loss: 0.7101     
Epoch 2/20
13892/13892 [==============================] - 5s - loss: 0.6859     
Epoch 3/20
13892/13892 [==============================] - 5s - loss: 0.6803     
Epoch 4/20
13892/13892 [==============================] - 6s - loss: 0.6773     
Epoch 5/20
13892/13892 [==============================] - 4s - loss: 0.6751     
Epoch 6/20
13892/13892 [==============================] - 4s - loss: 0.6736     
Epoch 7/20
13892/13892 [==============================] - 4s - loss: 0.6719     
Epoch 8/20
13892/13892 [==============================] - 4s - loss: 0.6706     
Epoch 9/20
13892/13892 [==============================] - 5s - loss: 0.6691     
Epoch 10/20
13892/13892 [==============================] - 5s - loss: 0.6681     
Epoch 11/20
13892/13892 [==============================] - 6s - loss: 0.6669     
Epoch 12/20
13892/13892 [==============================] - 6s - loss: 0.6659     
Epoch 13/20
13892/13892 [

In [13]:
def classifier(d):
    num_to_remove = 3
    for i in range(num_to_remove):
        d.pop()
    d.add(Dense(128, input_dim=64, activation='tanh', bias=True))
    d.add(Dense(128, activation='tanh', bias=True))
    d.add(Dense(num_classes, activation='softmax', bias=True))
    d.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
    return d

In [14]:
c = classifier(d)

c:\python\python35\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, use_bias=True, input_dim=64, activation="tanh")`
  """
c:\python\python35\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, use_bias=True, activation="tanh")`
  
c:\python\python35\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(13, use_bias=True, activation="softmax")`
  import sys


In [15]:
c.fit(train_X, train_y, validation_data=(val_X, val_y), nb_epoch=nb_epochs, batch_size=batch_size)

c:\python\python35\lib\site-packages\keras\models.py:851: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 13892 samples, validate on 6038 samples
Epoch 1/20
13892/13892 [==============================] - 4s - loss: 0.3001 - acc: 0.8947 - val_loss: 0.2098 - val_acc: 0.9232
Epoch 2/20
13892/13892 [==============================] - 4s - loss: 0.1385 - acc: 0.9526 - val_loss: 0.1812 - val_acc: 0.9374
Epoch 3/20
13892/13892 [==============================] - 4s - loss: 0.1078 - acc: 0.9647 - val_loss: 0.2036 - val_acc: 0.9357
Epoch 4/20
13892/13892 [==============================] - 4s - loss: 0.0882 - acc: 0.9708 - val_loss: 0.1518 - val_acc: 0.9546
Epoch 5/20
13892/13892 [==============================] - 4s - loss: 0.0791 - acc: 0.9742 - val_loss: 0.1428 - val_acc: 0.9558
Epoch 6/20
13892/13892 [==============================] - 4s - loss: 0.0707 - acc: 0.9763 - val_loss: 0.1354 - val_acc: 0.9599
Epoch 7/20
13892/13892 [==============================] - 4s - loss: 0.0659 - acc: 0.9783 - val_loss: 0.1298 - val_acc: 0.9622
Epoch 8/20
13892/13892 [==============================] - 4s -

In [16]:
loss, acc = c.evaluate(test_AP_features, test_labels)

  32/1111 [..............................] - ETA: 0s

In [18]:
print (loss, acc)

1.51719259608 0.735373537568
